# Deploy comunication compression scheme in FedLab

This tutorial provides comprehensive examples about implementing a communication efficiency scheme in FedLab. 

We take the baseline gradient compression algorithms as examples (top-k for gradient sparsification and QSGD for gradient quantization).

## Compress example

In [2]:
import sys
sys.path.append("../")

from fedlab.contrib.compressor.quantization import QSGDCompressor
from fedlab.contrib.compressor.topk import TopkCompressor
import torch

tpk_compressor = TopkCompressor(compress_ratio=0.05) # top 5% gradient
qsgd_compressor = QSGDCompressor(n_bit=8)


In [3]:
# top-k
tensor = torch.randn(size=(100,))
shape = tensor.shape
print("To be compressed tensor:", tensor)

# compress
values, indices = tpk_compressor.compress(tensor)
print("Compressed results top-k values:",values)
print("Compressed results top-k indices:", indices)

# decompress
decompressed = tpk_compressor.decompress(values, indices, shape)
print("Decompressed results:", decompressed)

To be compressed tensor: tensor([ 6.1734e-01,  5.4566e-01, -2.7149e-01,  6.5477e-01, -1.5721e-01,
         7.6196e-01,  3.2228e-02, -7.1947e-02,  2.0246e-03, -3.1017e-01,
        -5.2364e-02,  1.3370e-02, -1.2945e+00, -8.5691e-01, -1.1126e+00,
         8.4756e-02,  8.9776e-02, -7.6016e-02,  3.1314e-01,  1.9725e+00,
         3.3934e-01, -9.9729e-01,  1.0586e+00, -9.6778e-01, -7.4282e-01,
         6.4650e-01,  6.4144e-01,  9.5873e-01,  1.0628e+00, -1.4499e-01,
        -4.6418e-01, -4.1658e-01, -1.9156e-01, -1.8017e+00,  9.3635e-01,
        -3.1408e-01, -5.3132e-01,  5.2965e-01,  1.8596e-01, -1.2275e+00,
        -1.1480e-01, -1.1655e+00, -2.0570e+00,  5.1458e-02, -3.8844e-01,
        -8.8315e-01, -9.8421e-01,  1.4734e-01, -4.4630e-01, -3.2552e-01,
         4.4482e-01,  3.5355e-01, -8.5873e-02,  2.3461e-01, -1.8525e+00,
         2.1844e+00, -9.5873e-01, -6.0806e-01, -1.9722e+00, -7.7158e-01,
         1.3210e+00,  3.6783e-02, -1.7248e+00, -4.4206e-01, -6.6339e-01,
         1.4954e+00,  5.86

In [4]:
# qsgd
tensor = torch.randn(size=(100,))
shape = tensor.shape
print("To be compressed tensor:", tensor)

# compress
norm, signs, values = qsgd_compressor.compress(tensor)
print("Compressed results QSGD norm:", norm)
print("Compressed results QSGD signs:", signs)
print("Compressed results QSGD values:", values)


To be compressed tensor: tensor([-0.5874,  0.1735, -1.2135,  0.1112,  0.5729,  0.7815,  0.4815, -1.0461,
        -0.9385,  0.6850, -1.4863,  0.5474, -0.0624,  0.6318, -1.2670,  0.5215,
         0.3883,  0.0285, -0.8940,  0.4592,  0.5786,  1.7753,  1.2983,  1.3186,
         0.0944,  0.6858,  0.4261, -0.9178, -1.1772, -1.2924, -1.0322, -1.6536,
         1.6099,  0.1201,  1.0224,  0.9428, -0.8205, -0.3457, -1.8156, -1.0902,
         0.8046, -1.3716, -1.5002, -0.1688,  0.5523, -0.5052, -0.9781,  1.8258,
        -0.1122, -1.0995, -0.4280,  0.5172,  0.2979,  0.8120,  0.0586,  1.3134,
         0.2217, -0.7003, -0.4524,  0.8423, -0.1866, -0.4635,  0.9709,  1.3031,
         0.3019,  0.5531, -0.8191,  0.4463,  2.4395,  0.4004,  0.0116, -0.6406,
         0.2261,  0.4500, -0.8234, -0.3746,  0.5307,  0.3020, -1.5407,  0.9442,
         0.2261, -0.4509,  1.4290, -0.6274,  0.7533,  0.6486, -1.7730, -0.1257,
         0.0763, -0.2843, -1.0591,  0.0258, -1.1020, -0.9979, -0.9123, -0.1618,
         1.8885

In [5]:
# decompress
decompressed = qsgd_compressor.decompress([norm, signs, values])
print("Decompressed results:", decompressed)

Decompressed results: tensor([-0.5908,  0.1715, -1.2198,  0.1144,  0.5718,  0.7814,  0.4765, -1.0482,
        -0.9339,  0.6861, -1.4866,  0.5527, -0.0667,  0.6289, -1.2674,  0.5241,
         0.3907,  0.0286, -0.8958,  0.4574,  0.5813,  1.7820,  1.2960,  1.3151,
         0.0953,  0.6861,  0.4288, -0.9148, -1.1817, -1.2960, -1.0292, -1.6486,
         1.6105,  0.1239,  1.0197,  0.9434, -0.8291, -0.3431, -1.8201, -1.0959,
         0.8100, -1.3722, -1.5057, -0.1715,  0.5527, -0.5051, -0.9815,  1.8201,
        -0.1048, -1.0959, -0.4288,  0.5146,  0.2954,  0.8100,  0.0667,  1.3151,
         0.2192, -0.7052, -0.4479,  0.8481, -0.1811, -0.4669,  0.9720,  1.2960,
         0.3049,  0.5527, -0.8195,  0.4479,  2.4395,  0.4002,  0.0095, -0.6385,
         0.2287,  0.4479, -0.8291, -0.3812,  0.5337,  0.3049, -1.5438,  0.9434,
         0.2287, -0.4479,  1.4294, -0.6289,  0.7528,  0.6480, -1.7725, -0.1239,
         0.0762, -0.2859, -1.0578,  0.0286, -1.1054, -1.0006, -0.9148, -0.1620,
         1.8868, -

## Use compressor in federated learning

For example on the client side, we could compress the tensors are to compressed and upload the compressed results to server. And server could decompress the tensors follows the compression agreements.

In jupyter notebook, we take the standalone scenario as example.

In [6]:
from fedlab.contrib.algorithm.basic_client import SGDSerialClientTrainer, SGDClientTrainer
from fedlab.contrib.algorithm.basic_server import SyncServerHandler

class CompressSerialClientTrainer(SGDSerialClientTrainer):
    def setup_compressor(self, compressor):
        #self.compressor = TopkCompressor(compress_ratio=k)
        self.compressor = compressor

    @property
    def uplink_package(self):
        package = super().uplink_package
        new_package = []
        for content in package:
            pack = [self.compressor.compress(content[0])]
            new_package.append(pack)
        return new_package

class CompressServerHandeler(SyncServerHandler):
    def setup_compressor(self, compressor, type):
        #self.compressor = TopkCompressor(compress_ratio=k)
        self.compressor = compressor
        self.type = type

    def load(self, payload) -> bool:
        if self.type == "topk":
            values, indices = payload[0]
            decompressed_payload = self.compressor.decompress(values, indices, self.model_parameters.shape)

        if self.type == "qsgd":
            n, s, l = payload[0]
            decompressed_payload = self.compressor.decompress((n,s,l))
        
        return super().load([decompressed_payload])

In [8]:
# main, this part we follow the pipeline in pipeline_tutorial.ipynb
# But replace the hander and trainer by the above defined for communication compression

# configuration
from opcode import cmp_op
from munch import Munch
from fedlab.models.mlp import MLP

model = MLP(784, 10)
args = Munch

args.total_client = 100
args.alpha = 0.5
args.seed = 42
args.preprocess = False
args.cuda = True
args.cmp_op = "qsgd" # "topk, qsgd"

args.k = 0.1 # topk
args.bit = 8 # qsgd

if args.cmp_op == "topk":
    compressor = TopkCompressor(args.k)

if args.cmp_op == "qsgd":
    compressor = QSGDCompressor(args.bit)

from torchvision import transforms
from fedlab.contrib.dataset.partitioned_mnist import PartitionedMNIST

fed_mnist = PartitionedMNIST(root="../datasets/mnist/",
                             path="../datasets/mnist/fedmnist/",
                             num_clients=args.total_client,
                             partition="noniid-labeldir",
                             dir_alpha=args.alpha,
                             seed=args.seed,
                             preprocess=args.preprocess,
                             download=True,
                             verbose=True,
                             transform=transforms.Compose([
                                 transforms.ToPILImage(),
                                 transforms.ToTensor()
                             ]))

dataset = fed_mnist.get_dataset(0)  # get the 0-th client's dataset
dataloader = fed_mnist.get_dataloader(
    0,
    batch_size=128)  # get the 0-th client's dataset loader with batch size 128


In [9]:
# client
from fedlab.contrib.algorithm.basic_client import SGDSerialClientTrainer, SGDClientTrainer

# local train configuration
args.epochs = 5
args.batch_size = 128
args.lr = 0.1

trainer = CompressSerialClientTrainer(model, args.total_client,
                                 cuda=args.cuda)  # serial trainer
# trainer = SGDClientTrainer(model, cuda=True) # single trainer

trainer.setup_dataset(fed_mnist)
trainer.setup_optim(args.epochs, args.batch_size, args.lr)
trainer.setup_compressor(compressor)

# server
from fedlab.contrib.algorithm.basic_server import SyncServerHandler

# global configuration
args.com_round = 10
args.sample_ratio = 0.1

handler = CompressServerHandeler(model=model,
                            global_round=args.com_round,
                            sample_ratio=args.sample_ratio,
                            cuda=args.cuda)
handler.setup_compressor(compressor, args.cmp_op)

In [11]:
from fedlab.utils.functional import evaluate
from fedlab.core.standalone import StandalonePipeline

from torch import nn
from torch.utils.data import DataLoader
import torchvision

class EvalPipeline(StandalonePipeline):
    def __init__(self, handler, trainer, test_loader):
        super().__init__(handler, trainer)
        self.test_loader = test_loader

    def main(self):
        while self.handler.if_stop is False:
            # server side
            sampled_clients = self.handler.sample_clients()
            broadcast = self.handler.downlink_package

            # client side
            self.trainer.local_process(broadcast, sampled_clients)
            uploads = self.trainer.uplink_package

            # server side
            for pack in uploads:
                self.handler.load(pack)

            loss, acc = evaluate(self.handler.model, nn.CrossEntropyLoss(),
                                 self.test_loader)
            print("loss {:.4f}, test accuracy {:.4f}".format(loss, acc))


test_data = torchvision.datasets.MNIST(root="../datasets/mnist/",
                                       train=False,
                                       transform=transforms.ToTensor())
test_loader = DataLoader(test_data, batch_size=1024)

standalone_eval = EvalPipeline(handler=handler,
                               trainer=trainer,
                               test_loader=test_loader)
standalone_eval.main()

loss 21.9179, test accuracy 0.1019
loss 17.1236, test accuracy 0.5314
loss 13.0101, test accuracy 0.5469
loss 10.5621, test accuracy 0.6255
loss 8.4762, test accuracy 0.6965
loss 6.7011, test accuracy 0.7962
loss 6.2239, test accuracy 0.8110
loss 5.0510, test accuracy 0.8632
loss 4.5892, test accuracy 0.8714
loss 4.9932, test accuracy 0.8484
